In [1]:
import pandas as pd
import numpy as np

# ==============================
# 1. 데이터 로드
# ==============================
df = pd.read_csv("data/driving_log.csv")

# ==============================
# 2. 임계값 설정 (m/s^2)
# ==============================
HARD_ACCEL_THRESHOLD = 3.0
HARD_BRAKE_THRESHOLD = -3.0

# ==============================
# 3. 급가속 / 급제동 이벤트 탐지
# ==============================
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]

hard_accel_count = len(hard_accel_df)
hard_brake_count = len(hard_brake_df)
total_hard_events = hard_accel_count + hard_brake_count

# ==============================
# 4. 주행 시간 / 거리 계산
# ==============================
total_time_sec = df["time_sec"].max() - df["time_sec"].min()
avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600)

# ==============================
# 5. 이벤트 빈도 계산 (안정성 처리)
# ==============================
if total_time_sec >= 60:
    events_per_min = total_hard_events / (total_time_sec / 60)
else:
    events_per_min = None

if distance_km > 0.5:
    events_per_km = total_hard_events / distance_km
else:
    events_per_km = None

# ==============================
# 6. 분류 함수
# ==============================
def classify_by_count(total_events):
    if total_events <= 2:
        return "Smooth Driver"
    elif total_events <= 5:
        return "Normal Driver"
    else:
        return "Aggressive Driver"


def classify_by_rate(events_per_min):
    if events_per_min is None:
        return "Insufficient Data"
    elif events_per_min < 3:
        return "Smooth Driver"
    elif events_per_min < 6:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

# ==============================
# 7. 분류 결과
# ==============================
driver_type_count = classify_by_count(total_hard_events)
driver_type_rate = classify_by_rate(events_per_min)

# ==============================
# 8. 최종 출력
# ==============================
print("Driver Behavior Analysis Result")
print("-" * 35)
print(f"Hard Acceleration Events : {hard_accel_count}")
print(f"Hard Braking Events      : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")

if events_per_min is not None:
    print(f"Events per Minute        : {events_per_min:.2f}")
else:
    print("Events per Minute        : Insufficient Data")

if events_per_km is not None:
    print(f"Events per km            : {events_per_km:.2f}")
else:
    print("Events per km            : Insufficient Data")

print("-" * 35)
print(f"Classification (Count)   : {driver_type_count}")
print(f"Classification (Rate)    : {driver_type_rate}")


Driver Behavior Analysis Result
-----------------------------------
Hard Acceleration Events : 3
Hard Braking Events      : 3
Total Hard Events        : 6
Total Time (sec)         : 9.0
Events per Minute        : Insufficient Data
Events per km            : Insufficient Data
-----------------------------------
Classification (Count)   : Aggressive Driver
Classification (Rate)    : Insufficient Data
